In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# Load the dataset
df = pd.read_csv('TRADES_CopyTr_90D_ROI.csv')

In [3]:
df.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [5]:
df.describe()

,Port_IDs
count,1.500000e+02
mean,3.975655e+18
std,6.887470e+16
min,3.672755e+18
25%,3.943672e+18
50%,3.998616e+18
75%,4.029508e+18
max,4.041860e+18


In [6]:
# Handle missing values
df.isnull().sum()
df.dropna(inplace=True)  # Drop rows with missing valuesdata.isnull().sum()

In [7]:
df.columns

Index(['Port_IDs', 'Trade_History'], dtype='object')

In [9]:
sub_columns = list['time','symbol','side','price','fee','feeAsset','quantity','quantityAsset','realizedProfit','realizedProfitAsset','baseAsset','qty','positionSide','activeBuy']
# Select only the valid columns
sub_df = df[sub_columns]
print(sub_df)

                Port_IDs                                      Trade_History
0    3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1    4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2    3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3    3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4    3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
..                   ...                                                ...
145  4000222729738650369  [{'time': 1718982068000, 'symbol': 'ARKMUSDT',...
146  3998659472131949824  [{'time': 1718979385000, 'symbol': 'BTCUSDT', ...
147  4028701921959171840  [{'time': 1718984241000, 'symbol': 'BTCUSDT', ...
148  4014818740371615232  [{'time': 1718983357000, 'symbol': 'SOLUSDT', ...
149  3768170840939476993  [{'time': 1718977395000, 'symbol': 'BNBUSDT', ...

[149 rows x 2 columns]


In [11]:
df.columns

Index(['Port_IDs', 'Trade_History'], dtype='object')

In [17]:
import ast
# Convert the Trade_History strings to actual lists of dictionaries
df['Trade_History'] = df['Trade_History'].apply(ast.literal_eval)

# Explode the Trade_History column to create a new DataFrame
df_exploded = df.explode('Trade_History')

# Extract the 'time' from the Trade_History
df_exploded['time'] = df_exploded['Trade_History'].apply(lambda x: x['time'])

# Convert timestamps to datetime
df_exploded['time'] = pd.to_datetime(df_exploded['time'], unit='ms')

# Display the final DataFrame
print(df_exploded[['Port_IDs', 'time']])

                Port_IDs                time
0    3925368433214965504 2024-06-20 16:07:36
0    3925368433214965504 2024-06-20 16:06:58
0    3925368433214965504 2024-06-20 16:06:58
0    3925368433214965504 2024-06-20 16:06:56
0    3925368433214965504 2024-06-20 16:06:56
..                   ...                 ...
149  3768170840939476993 2024-06-16 14:57:55
149  3768170840939476993 2024-06-16 14:57:55
149  3768170840939476993 2024-06-16 14:57:55
149  3768170840939476993 2024-06-16 14:57:55
149  3768170840939476993 2024-06-16 14:57:55

[211277 rows x 2 columns]


In [35]:
# Step 2: Feature Engineering
# Position Identification
df_exploded['trade_type'] = df_exploded['Trade_History'].apply(lambda x: f"{x['side']}_{x['positionSide']}")
df_exploded['quantity'] = df_exploded['Trade_History'].apply(lambda x: x.get('quantity')) 
df_exploded['price'] = df_exploded['Trade_History'].apply(lambda x: x.get('price'))  
df_exploded['realizedProfit'] = df_exploded['Trade_History'].apply(lambda x: x.get('realizedProfit'))
df_exploded['investment'] = df_exploded['Trade_History'].apply(lambda x: x.get('investment')) 

# Calculate investment amount
df_exploded['investment'] = df_exploded['quantity'] * df_exploded['price']

# Calculate daily returns for Sharpe Ratio
df_exploded['daily_return'] = df_exploded['realizedProfit'] / df_exploded['investment']

In [39]:
# Step 3: Calculate Financial Metrics
# Aggregate Metrics by Account
metrics = df_exploded.groupby('Port_IDs').agg(
    total_positions=('trade_type', 'count'),
    win_positions=('realizedProfit', lambda x: (x > 0).sum()),
    total_pnl=('realizedProfit', 'sum'),
    total_investment=('investment', 'sum'),
).reset_index()


In [41]:
# Calculate ROI and Win Rate
metrics['roi'] = (metrics['total_pnl'] / metrics['total_investment']) * 100
metrics['win_rate'] = (metrics['win_positions'] / metrics['total_positions']) * 100

In [47]:
# Calculate Sharpe Ratio
daily_returns = df_exploded.groupby(['Port_IDs', df_exploded['time'].dt.date])['daily_return'].sum()
sharpe_ratios = daily_returns.groupby('Port_IDs').apply(lambda x: (x.mean() / x.std()) if x.std() != 0 else 0)
metrics['sharpe_ratio'] = sharpe_ratios.values

In [49]:
# Calculate Maximum Drawdown (MDD)
def max_drawdown(returns):
    cumulative = (1 + returns).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    return drawdown.min()

mdd = daily_returns.groupby('Port_IDs').apply(max_drawdown)
metrics['mdd'] = mdd.values

In [51]:
# Step 4: Ranking Algorithm
# Scoring System
weights = {
    'roi': 0.4,
    'sharpe_ratio': 0.3,
    'win_rate': 0.3
}
metrics['score'] = (metrics['roi'] * weights['roi'] +
                    metrics['sharpe_ratio'] * weights['sharpe_ratio'] +
                    metrics['win_rate'] * weights['win_rate'])

# Rank Accounts
metrics['rank'] = metrics['score'].rank(ascending=False)

# Get Top 20 Accounts
top_20_accounts = metrics.sort_values(by='score', ascending=False).head(20)

In [53]:
# Display Results
print("\nTop 20 Accounts:")
print(top_20_accounts[['Port_IDs', 'total_positions', 'win_positions', 'total_pnl', 'roi', 'win_rate', 'sharpe_ratio', 'mdd', 'score', 'rank']])



Top 20 Accounts:
                Port_IDs  total_positions  win_positions     total_pnl  \
58   3977234346014419201               83             33   2427.292831   
1    3733192481840423936              689            553   2923.977200   
62   3986814617275053313             3554           2780  16337.461881   
16   3891020560590657281              437            283   2856.300564   
14   3886752488982104320             1249            931   7195.178325   
47   3956048468100538880               28             20   1373.564890   
88   4013955990069013504              358            236   1690.027050   
17   3907081197088384000             4137           2540  18015.997370   
8    3826087012661391104              108             63    532.656974   
61   3983074113875692800               43             14   2424.829520   
144  4039279455324236544              327            181   1038.807419   
92   4017110277719148289              252            143   2899.933021   
77   400022272973865

In [57]:
# Save the calculated metrics to a CSV file
metrics.to_csv('calculated_metrics.csv', index=False)

# Save the top 20 accounts to a CSV file
top_20_accounts.to_csv('top_20_accounts.csv', index=False)